In [1]:
import torch

In [2]:
z = torch.zeros(5,3)

In [5]:
print(z.dtype)

torch.float32


In [6]:
i = torch.ones((2,2),dtype=torch.int)
print(i)
print(i.dtype)

tensor([[1, 1],
        [1, 1]], dtype=torch.int32)
torch.int32


In [11]:
torch.manual_seed(1729)

t1 = torch.rand(2,2)
t2 = torch.rand(2,2)

torch.manual_seed(1729)
t3 = torch.rand(2,2)

print(t1)
print(t2)
print(t3)

tensor([[0.3126, 0.3791],
        [0.3087, 0.0736]])
tensor([[0.4216, 0.0691],
        [0.2332, 0.4047]])
tensor([[0.3126, 0.3791],
        [0.3087, 0.0736]])


In [12]:
if (t1==t3):
    print('yes equal')


RuntimeError: Boolean value of Tensor with more than one value is ambiguous

Basic Mechanics of  a single training pass
Autograd -- Automatic Differentiation Engine 
A model will have forward(), here the actual computation happens


# Define Neural network with activation and loss functions

class LeNet(nn.Module):
      def __init__(self):
         super(LeNet,self).__init__()
         self.conv1 = nn.Conv2d(1,6,3)
         self.fc1   = nn.Linear(84,10)

      def forward(self,x):
          x = F.max_pool2d(F.relu(self.conv1(x), (2,2)))
          x = F.relu(self.fc1(x))


# Create Object for the above model classs
net = LeNet() #Instatiate class
print(net)    

# Prepare Input for Inference
input = torch.rand(1,1,32,32)  # this tells that this pytorch input has 32*32 image with 1 input channel 
                               # and the first '1' represents its batch number
                               # eg., if the batch number is 16 then input will be (16,1,32,32)
                               # pytorch model assumes they are working on batches of data


# Inference
output = net(input) 




In [13]:
import torch                       #for all things pytorch
import torch.nn as nn              # for torch.nn.Module, the parent object for pytorch Models
import torch.nn.functional as F    #for Activation functions


In [4]:
# torch.utils.data.Dataset   all downloadable datasets are subclassess of torch.utils.data.Dataset class.
# Dataset classes in pytorch include the downloadable datasets in TrochVision, Trochtext and TorchAudio as well as utility dataset classes such as 
# torchvision.datasets.ImageFolder which reads imagers from folder

## Dataset -------> DataLoader 


import torch
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])


trainset = torchvision.datasets.CIFAR10(root='./data',train=True,download=True, transform = transform)


170499072it [00:19, 8754376.04it/s]                                


Extracting ./data\cifar-10-python.tar.gz to ./data


In [5]:
# Once our dataset is ready we can give it to DataLoader
trainloader = torch.utils.data.DataLoader(trainset, batch_size = 4, shuffle=True, num_workers=2)

In [6]:
## Putting all pieces together

import torch
import torch.nn as nn              # For Model
import torch.nn.functional as F    # For Activation Function
import torch.optim as optim        # For 

import torchvision                 # For Computer Vision 
import torchvision.transforms as transforms   # transformation function 

import numpy as np


#getting data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])


# Training Data
trainset = torchvision.datasets.CIFAR10(root='./data',train=True,download=True, transform = transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size = 4, shuffle=True, num_workers=2)

#Testing Data
testset = torchvision.datasets.CIFAR10(root='./data',train=False,download=True, transform = transform)

testloader = torch.utils.data.DataLoader(testset, batch_size = 4, shuffle=False, num_workers=2)

classes = ('plane','car','bird','cat','deer','dog','frog','horse','ship','truck')


Files already downloaded and verified
Files already downloaded and verified


In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d(3,6,5)
        self.pool  = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6,16,5)
        self.fc1   = nn.Linear(16*5*5,120)
        self.fc2   = nn.Linear(120,84)
        self.fc3   = nn.Linear(84,10)

    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1,16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()        


In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(),lr=0.001,momentum=0.9)


In [9]:
## Training

for epoch in range(2):  # run for 2 epoches
    running_loss = 0.0
    for i, data in enumerate(trainloader,0):
        #get the inputs
        inputs, labels = data

        #zero the parameter gradients
        optimizer.zero_grad()   #zero gradients for every batch

        #forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()

        #print stats
        running_loss += loss.item()
        if i%2000 == 1999: #print every 2000 mini batches
            print('[%d, %5d] loss:%.3f' %(epoch+1, i+1, running_loss / 2000))
            running_loss = 0.0 

print('finished training')
 



[1,  2000] loss:2.179
[1,  4000] loss:1.846
[1,  6000] loss:1.687
[1,  8000] loss:1.608
[1, 10000] loss:1.568
[1, 12000] loss:1.507
[2,  2000] loss:1.448
[2,  4000] loss:1.414
[2,  6000] loss:1.382
[2,  8000] loss:1.370
[2, 10000] loss:1.333
[2, 12000] loss:1.327
finished training


In [10]:
# Testing
correct = 0
total = 0

with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data,1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"accuracy is {100 * correct/ total}") 


accuracy is 52.74


In [16]:
# Deployement with TorchScript

my_module = torch.jit.script(net)

NameError: name 'x' is not defined

In [12]:
my_module

RecursiveScriptModule(
  original_name=Net
  (conv1): RecursiveScriptModule(original_name=Conv2d)
  (pool): RecursiveScriptModule(original_name=MaxPool2d)
  (conv2): RecursiveScriptModule(original_name=Conv2d)
  (fc1): RecursiveScriptModule(original_name=Linear)
  (fc2): RecursiveScriptModule(original_name=Linear)
  (fc3): RecursiveScriptModule(original_name=Linear)
)

In [67]:
from transformers import pipeline

classifier = pipeline("ner",grouped_entities=True)
classifier("I've not been waiting for a HuggingFace course my whole life, this is not going to be work.")

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english)
c:\Program Files\Python39\lib\site-packages\transformers\pipelines\token_classification.py:135: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.SIMPLE"` instead.
  warnings.warn(


[{'entity_group': 'MISC',
  'score': 0.8987888,
  'word': 'HuggingFace',
  'start': 28,
  'end': 39}]

In [82]:
classifier("Raghu went to hyderabad for dental surgery at X-RAY Inc")

[{'entity_group': 'PER',
  'score': 0.9294071,
  'word': 'Raghu',
  'start': 0,
  'end': 5},
 {'entity_group': 'LOC',
  'score': 0.8643474,
  'word': 'hyderabad',
  'start': 14,
  'end': 23},
 {'entity_group': 'ORG',
  'score': 0.9946643,
  'word': 'X - RAY Inc',
  'start': 46,
  'end': 55}]

In [10]:
#Zero Short Classifier

from transformers import pipeline
 
zero_classifier = pipeline("zero-shot-classification")

No model was supplied, defaulted to facebook/bart-large-mnli (https://huggingface.co/facebook/bart-large-mnli)
Downloading: 100%|██████████| 1.13k/1.13k [00:00<00:00, 230kB/s]
Downloading: 100%|██████████| 1.52G/1.52G [00:53<00:00, 30.3MB/s]
Downloading: 100%|██████████| 26.0/26.0 [00:00<00:00, 5.20kB/s]
Downloading: 100%|██████████| 878k/878k [00:01<00:00, 659kB/s]  
Downloading: 100%|██████████| 446k/446k [00:01<00:00, 376kB/s]  
Downloading: 100%|██████████| 1.29M/1.29M [00:01<00:00, 971kB/s]


In [66]:
zero_classifier(
    "Cosentyx marketing campaign",
    candidate_labels=[ "Creative Agency","Public Relations","Medical Communications","Primary market Research & Competitive Intelligence",
    "Secondary Market Research", "Health Economics and Access Pricing","chemicals"],
)

{'sequence': 'Cosentyx marketing campaign',
 'labels': ['Public Relations',
  'chemicals',
  'Secondary Market Research',
  'Creative Agency',
  'Medical Communications',
  'Primary market Research & Competitive Intelligence',
  'Health Economics and Access Pricing'],
 'scores': [0.5262454152107239,
  0.2862624526023865,
  0.045808836817741394,
  0.0424041822552681,
  0.03814248368144035,
  0.0307435542345047,
  0.030393054708838463]}

Using Tokenizers and Models Explicitly for Pretrained Models

In [1]:
from transformers import AutoTokenizer


In [2]:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

model_inputs = tokenizer(sequence)

In [3]:
model_inputs

{'input_ids': [101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [5]:
# tokenizer It also handles multiple sequences at a time, with no change in the API:

sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]

model_inputs = tokenizer(sequences)
model_inputs

{'input_ids': [[101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102], [101, 2061, 2031, 1045, 999, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

In [11]:
#It can pad according to several objectives:
# print(f'max length is {max_length} according to the tokenizer that we use')
model_inputs = tokenizer(sequences, padding="longest")
# model_inputs = tokenizer(sequences, padding="max_length")
# model_inputs = tokenizer(sequences, padding="max_length", max_length=8)

print(model_inputs)


#It can also truncate sequences:
sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]
# Will truncate the sequences that are longer than the model max length
# (512 for BERT or DistilBERT)
model_inputs = tokenizer(sequences, truncation=True)
print(model_inputs)

# Will truncate the sequences that are longer than the specified max length
model_inputs = tokenizer(sequences, max_length=8, truncation=True)
print(model_inputs)



{'input_ids': [[101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102], [101, 2061, 2031, 1045, 999, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]}
{'input_ids': [[101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102], [101, 2061, 2031, 1045, 999, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}
{'input_ids': [[101, 1045, 1005, 2310, 2042, 3403, 2005, 102], [101, 2061, 2031, 1045, 999, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}


In [12]:
print(tokenizer.tokenize(sequences))

['i', "'", 've', 'been', 'waiting', 'for', 'a', 'hugging', '##face', 'course', 'my', 'whole', 'life', '.', 'so', 'have', 'i', '!']
